In [1]:
from ultralytics import YOLO
import torch
import sparseml
import torchvision
from sparseml.pytorch.optim import ScheduledModifierManager
from sparseml.pytorch.utils import TensorBoardLogger, ModuleExporter, get_prunable_layers, tensor_sparsity
from torch.utils.data import DataLoader
from torch.nn import CrossEntropyLoss
from torch.optim import Adam
from torchvision import transforms
from tqdm.auto import tqdm
import math
import datasets

In [2]:
dense_recipe_path = "recipes/yolov11n-cocoa-dense-recipe.yaml"

In [ ]:
# Device configuration
device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)

def run_yolo_one_epoch(model, data_loader, device, train=False, optimizer=None):
    if train:
        model.train()
    else:
        model.eval()

    running_loss = 0.0
    total_correct = 0
    total_predictions = 0

    # Loop through batches
    for step, (images, targets) in tqdm(enumerate(data_loader), total=len(data_loader)):
        images = images.to(device)
        targets = [{k: v.to(device) for k, v in t.items()} for t in targets]  # move target data to the device

        if train:
            optimizer.zero_grad()

        # Forward pass
        outputs = model(images)

        # Compute loss (YOLO models use a built-in loss during training)
        loss_dict = model.compute_loss(outputs, targets)  # Custom method for YOLO
        loss = sum(loss_dict.values())  # Combine all losses

        if train:
            loss.backward()
            optimizer.step()

        running_loss += loss.item()

        # Evaluation (e.g., counting correct predictions)
        for output, target in zip(outputs, targets):
            preds = output["boxes"]  # Example: Access predicted bounding boxes
            labels = target["boxes"]
            # This is an example. Actual metrics depend on your evaluation criteria.

    # Return average loss (YOLO accuracy metrics are more complex)
    loss = running_loss / (step + 1.0)
    return loss

# Example usage
# Load a YOLO model and a DataLoader
model = YOLO("yolo11n.pt").to(device)

# Dummy DataLoader for illustration (replace with your actual data loader)
from torch.utils.data import DataLoader
from torchvision.datasets import FakeData
from torchvision.transforms import ToTensor

fake_loader = DataLoader(FakeData(transform=ToTensor()), batch_size=4)

# Run the function
loss = run_yolo_one_epoch(model, fake_loader, device, train=True, optimizer=torch.optim.Adam(model.parameters()))
print(f"Loss: {loss}")


In [2]:
model = YOLO('runs/detect/train/weights/best.pt')


criterion = CrossEntropyLoss()
optimizer = Adam(model.parameters(), lr=8e-3) # lr will be override by sparseml

In [4]:
metrics=model.val(data="datasets/cocoa_diseases/cocoa_dataset.yaml",
                 batch=2,
                 device=[0,1])

Ultralytics 8.3.54 🚀 Python-3.8.10 torch-1.13.1+cu116 CUDA:0 (Tesla T4, 14918MiB)
                                                       CUDA:1 (Tesla T4, 14918MiB)


val: Scanning /home/jovyan/ML2/datasets/cocoa_diseases/labels/val.cache... 62 images, 0 backgrounds, 0 corrupt: 100%|██████████| 62/62 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 31/31 [00:02<00:00, 11.74it/s]


                   all         62        289       0.68      0.649      0.685      0.485
          phytophthora         30         39      0.584      0.564      0.592      0.448
               monilia         23         30      0.807      0.557      0.694      0.535
               healthy         52        220       0.65      0.826      0.768      0.471
Speed: 0.5ms preprocess, 11.3ms inference, 0.0ms loss, 0.9ms postprocess per image
Results saved to runs/detect/val3


In [10]:
# create ScheduledModifierManager and Optimizer wrapper
manager = ScheduledModifierManager.from_yaml(dense_recipe_path)
optimizer = manager.modify(model, optimizer, steps_per_epoch=len(train_loader))

NameError: name 'train_loader' is not defined

In [17]:
print(dir(model))

['T_destination', '__annotations__', '__call__', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattr__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__setstate__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_apply', '_backward_hooks', '_buffers', '_call_impl', '_check_is_pytorch_model', '_forward_hooks', '_forward_pre_hooks', '_get_backward_hooks', '_get_name', '_is_full_backward_hook', '_load', '_load_from_state_dict', '_load_state_dict_post_hooks', '_load_state_dict_pre_hooks', '_maybe_warn_non_full_backward_hook', '_modules', '_named_members', '_new', '_non_persistent_buffers_set', '_parameters', '_register_load_state_dict_pre_hook', '_register_state_dict_hook', '_replicate_for_data_parallel', '_reset_ckpt_args', '_save_to_state_dict', '_slow_forward', '_smart_load', '

In [26]:
print(model.children)

<bound method Module.children of YOLO(
  (model): DetectionModel(
    (model): Sequential(
      (0): Conv(
        (conv): Conv2d(3, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(16, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
      (1): Conv(
        (conv): Conv2d(16, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
      (2): C3k2(
        (cv1): Conv(
          (conv): Conv2d(32, 32, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
          (act): SiLU(inplace=True)
        )
        (cv2): Conv(
          (conv): Conv2d(48, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn): BatchNorm2d(64, eps=0.001, momentum=

In [ ]:
# Load the full model



model
# Use the model
train_results = model.train(
    data="datasets/cocoa_diseases/cocoa_dataset.yaml",
    epochs=5,
    project="cocoa_diseases_yolo11n",
    imgsz=640,
    batch=2,  # Small batch size
    device=[0,1],  # GPU
    workers=1  # Minimal number of workers
)